In [40]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
#!conda install matplotlib
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


In [2]:
dfs = pd.read_html('http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df = dfs[0]
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


"Ignore cells with a borough that is Not assigned">> I droped them:

In [3]:
df.drop(df.loc[df['Borough']=='Not assigned'].index,inplace=True)

"If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough">> didn't found any but wrote this code anyway:

In [4]:
df['Neighborhood'].loc[df['Neighborhood']=='Not assigned']=df['Borough']

In [5]:
df.shape

(103, 3)

Merging Latitude and longtitude data:

In [6]:
lon_lat=pd.read_csv('https://cocl.us/Geospatial_data')

In [7]:
df=df.join(lon_lat.set_index('Postal Code'), on='Postal Code')

In [9]:
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


Only Toronto Neighborhoods:

In [10]:
toronto=df.loc[df['Borough'].str.contains('Toronto', regex=False)==True]

Using foursquare

In [11]:
CLIENT_ID = '5DAL5IKIGQL4TQTNPEATC2J4YIFI2BTGEFQAU0L45NWWAFLU' # your Foursquare ID
CLIENT_SECRET = 'OFAEXC1MUT0IEHVCMM2HIVZI02C4CNMZYJY5PX3AC2RE3IAJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [49]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [50]:
LIMIT=100
radius=500
toronto_venues = getNearbyVenues(names=toronto['Neighborhood'],
                                   latitudes=toronto['Latitude'],
                                   longitudes=toronto['Longitude']
                                  )

toronto_venues.head()

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Corktown Common,43.655618,-79.356211,Park


In [53]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hospital,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

I've decided only to cluster food palces:

In [99]:
rest_cols = [col for col in toronto_onehot.columns if 'Restaurant' in col]
rest_cols.insert(0,'Neighborhood')
rest_cols.insert(1,'Pizza Place')
rest_cols.insert(1,'Burger Joint')
rest_cols.insert(1,'Burrito Place')
rest_cols.insert(1,'Bistro')
rest_cols.insert(1,'Breakfast Spot')
rest_cols.insert(1,'Bakery')
rest_cols.insert(1,'Bagel Shop')
rest_cols.insert(1,'BBQ Joint')
rest_cols.insert(1,'Coffee Shop')
rest_cols.insert(1,'Food Truck')
rest_cols.insert(1,'Fried Chicken Joint')






toronto_rest=toronto_onehot[rest_cols]


In [107]:
toronto_grouped = toronto_rest.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Fried Chicken Joint,Food Truck,Coffee Shop,BBQ Joint,Bagel Shop,Bakery,Breakfast Spot,Bistro,Burrito Place,Burger Joint,Pizza Place,American Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Colombian Restaurant,Comfort Food Restaurant,Cuban Restaurant,Doner Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,Ramen Restaurant,Restaurant,Seafood Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Berczy Park,0.000000,0.000000,0.086207,0.017241,0.017241,0.034483,0.017241,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.017241,0.000000,0.000000,0.017241,0.00,0.000000,0.000000,0.000000,0.017241,0.000000,0.000000,0.017241,0.017241,0.000000,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.034483,0.034483,0.017241,0.000000,0.017241,0.000000,0.017241,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.086957,0.000000,0.000000,0.043478,0.086957,0.000000,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.000000,0.171429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.028571,0.014286,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.014286,0.000000,0.000000,0.014286,0.000000,0.000000,0.000000,0.014286,0.071429,0.042857,0.014286,0.000000,0.000000,0.000000,0.014286,0.014286,0.000000,0.00000,0.014286,0.014286,0.000000,0.014286,0.014286,0.000000,0.028571,0.000000,0.014286,0.000000
5,Christie,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.000000,0.000000,0.084

running k-means:

In [108]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 1, 2, 0, 2, 2, 2, 2, 1])

adding cluster label to our neighborhoods

In [109]:
toronto_grouped.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = toronto.join(toronto_grouped.set_index('Neighborhood'), on='Neighborhood')


In [110]:
toronto_merged.head(20)

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,Fried Chicken Joint,Food Truck,Coffee Shop,BBQ Joint,Bagel Shop,Bakery,Breakfast Spot,Bistro,Burrito Place,Burger Joint,Pizza Place,American Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Colombian Restaurant,Comfort Food Restaurant,Cuban Restaurant,Doner Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,Ramen Restaurant,Restaurant,Seafood Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,0.000000,0.000000,0.177778,0.000000,0.000000,0.066667,0.044444,0.000000,0.000000,0.000000,0.000000,0.000000,0.022222,0.000000,0.000000,0.0,0.00000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.022222,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022222,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.022222,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,0.032258,0.000000,0.290323,0.000000,0.000000,0.000000,0.000000,0.000000,0.032258,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.032258,0.000000,0.000000,0.000000,0.000000,0.032258,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.032258,0.0,0.000000,0.0,0.000000,0.000000
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,0.000000,0.000000,0.080000,0.000000,0.000000,0.010000,0.000000,0.000000,0.010000,0.010000,0.020000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.01,0.000000,0.000000,0.000000,0.0,0.000000,0.01,0.000000,0.020000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.030000,0.000000,0.000000,0.000000,0.010000,0.020000,0.010000,0.0,0.00000,0.010000,0.020000,0.010000,0.010000,0.000000,0.0,0.010000,0.0,0.000000,0.010000
22,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,0.000000,0.011905,0.059524,0.011905,0.000000,0.011905,0.011905,0.011905,0.011905,0.000000,0.000000,0.035714,0.011905,0.011905,0.000000,0.0,0.00000,0.00,0.000000,0.011905,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.011905,0.011905,0.000000,0.000000,0.000000,0.023810,0.023810,0.000000,0.000000,0.000000,0.000000,0.011905,0.000000,0.0,0.02381,0.011905,0.000000,0.059524,0.023810,0.000000,0.0,0.011905,0.0,0.011905,0.000000
30,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
31,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,2,0.000000,0.000000,0.086207,0.017241,0.017241,0.034483,0.017241,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.00,0.000000,0.017241,0.000000,0.0,0.017241,0.00,0.000000,0.000000,0.0,0.017241,0.000000,0.000000,0.017241,0.017241,0.000000,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.034483,0.034483,0.017241,

Mapping the clusters:

In [111]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [162]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Cluster 0 Top 5 restaurants by freq>> lots of coffee shops

In [148]:
#toronto_merged.groupby('Cluster Labels').mean().transpose().drop(['Latitude','Longitude'],0)
cluster0=pd.DataFrame(toronto_merged.loc[toronto_merged['Cluster Labels']==0].mean().transpose().drop(['Latitude','Longitude','Cluster Labels'],0))
cluster0.head(5).rename(columns={0:'avg freq'}).sort_values(by=['avg freq'],ascending=False)

,avg freq
Coffee Shop,0.230876
Fried Chicken Joint,0.016129
Food Truck,0.000000
BBQ Joint,0.000000
Bagel Shop,0.000000


Cluster 1 Top 5 restaurants by freq>> fewer coffee shops

In [154]:
cluster1=pd.DataFrame(toronto_merged.loc[toronto_merged['Cluster Labels']==1].mean().transpose().drop(['Latitude','Longitude','Cluster Labels'],0))
cluster1.head(5).rename(columns={0:'avg freq'}).sort_values(by=['avg freq'],ascending=False)

,avg freq
Coffee Shop,0.004319
Fried Chicken Joint,0.003333
Food Truck,0.000000
BBQ Joint,0.000000
Bagel Shop,0.000000


Cluster 2 Top 5 restaurants by freq>> diverse places

In [158]:
cluster2=pd.DataFrame(toronto_merged.loc[toronto_merged['Cluster Labels']==2].mean().transpose().drop(['Latitude','Longitude','Cluster Labels'],0))
cluster2.head(5).rename(columns={0:'avg freq'}).sort_values(by=['avg freq'],ascending=False)

,avg freq
Coffee Shop,0.090868
Bagel Shop,0.004393
Fried Chicken Joint,0.003854
BBQ Joint,0.003727
Food Truck,0.001342


Cluster 3 Top 5 restaurants by freq>> nowhere to eat

In [161]:
cluster3=pd.DataFrame(toronto_merged.loc[toronto_merged['Cluster Labels']==3].mean().transpose().drop(['Latitude','Longitude','Cluster Labels'],0))
cluster3.head(5).rename(columns={0:'avg freq'}).sort_values(by=['avg freq'],ascending=False)

,avg freq
Fried Chicken Joint,0.0
Food Truck,0.0
Coffee Shop,0.0
BBQ Joint,0.0
Bagel Shop,0.0
